##Population Data (API)

In [3]:
import requests
import pandas as pd

api_key = '9fdaf9347899189e1001e62895efb22e6d7471b8'
url = f"https://api.census.gov/data/2022/acs/acsse?get=NAME,K200101_001E&for=place:*&in=state:06&key={api_key}"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df.rename(columns={'K200101_001E': 'Population', 'NAME': 'City Name', 'state': 'State Code', 'place': 'Place Code'}, inplace=True)
    df['Population'] = pd.to_numeric(df['Population'])
    print(df)
else:
    print(f"Failed to retrieve data: {response.status_code}")

                          City Name  Population State Code Place Code
0     Agoura Hills city, California       20594         06      00394
1         Adelanto city, California       38781         06      00296
2          Alameda city, California       76039         06      00562
3           Albany city, California       19370         06      00674
4         Alhambra city, California       80264         06      00884
..                              ...         ...        ...        ...
351  Winter Gardens CDP, California       21958         06      85992
352       Woodland city, California       61454         06      86328
353      Yuba City city, California       69004         06      86972
354        Yucaipa city, California       54406         06      87042
355   Yucca Valley town, California       21969         06      87056

[356 rows x 4 columns]


In [4]:
import requests
import pandas as pd

api_key = '9fdaf9347899189e1001e62895efb22e6d7471b8'
zip_code = '93063'
url = f"https://api.census.gov/data/2022/acs/acs5/profile?get=NAME,DP05_0001E&for=zip%20code%20tabulation%20area:{zip_code}&key={api_key}"

response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data[1:], columns=data[0])
    df.rename(columns={'DP05_0001E': 'Population', 'NAME': 'ZIP Code'}, inplace=True)
    df['Population'] = pd.to_numeric(df['Population'])
    print(df)
else:
    print(f"Failed to retrieve data: {response.status_code}")


      ZIP Code  Population zip code tabulation area
0  ZCTA5 93063       57075                    93063


In [ ]:
import requests
import pandas as pd

api_key = '9fdaf9347899189e1001e62895efb22e6d7471b8'

def get_population(zip_code):
    url = f"https://api.census.gov/data/2022/acs/acs5/profile?get=NAME,DP05_0001E&for=zip%20code%20tabulation%20area:{zip_code}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            return {'ZIP Code': data[1][0], 'Population': int(data[1][1])}
    else:
        print(f"Failed to retrieve data for ZIP code {zip_code}: {response.status_code}")
        return None

# Iterate over all ZIP codes from 90001 to 96150
population_data = []
for zip_code in range(90001, 96150):
    population_info = get_population(str(zip_code))
    if population_info:
        population_data.append(population_info)

# Create DataFrame from the collected population data
df = pd.DataFrame(population_data)
print(df)


Failed to retrieve data for ZIP code 90009: 204
Failed to retrieve data for ZIP code 90030: 204
Failed to retrieve data for ZIP code 90050: 204
Failed to retrieve data for ZIP code 90051: 204
Failed to retrieve data for ZIP code 90053: 204
Failed to retrieve data for ZIP code 90054: 204
Failed to retrieve data for ZIP code 90055: 204
Failed to retrieve data for ZIP code 90060: 204
Failed to retrieve data for ZIP code 90070: 204
Failed to retrieve data for ZIP code 90072: 204
Failed to retrieve data for ZIP code 90074: 204
Failed to retrieve data for ZIP code 90075: 204
Failed to retrieve data for ZIP code 90076: 204
Failed to retrieve data for ZIP code 90078: 204
Failed to retrieve data for ZIP code 90080: 204
Failed to retrieve data for ZIP code 90081: 204
Failed to retrieve data for ZIP code 90082: 204
Failed to retrieve data for ZIP code 90083: 204
Failed to retrieve data for ZIP code 90084: 204
Failed to retrieve data for ZIP code 90085: 204
Failed to retrieve data for ZIP code 900

In [ ]:
df

In [ ]:
# Remove "ZCTA5" prefix from ZIP codes
df['ZIP Code'] = df['ZIP Code'].str.replace('ZCTA5 ', '')

# Rename the column to "Zip Code"
df.rename(columns={'ZIP Code': 'Zip Code'}, inplace=True)

# Save DataFrame to CSV file
df.to_csv('population_data.csv', index=False)


In [ ]:
unique_zipcodes = df['Zip Code'].nunique()

print("Number of unique ZIP codes:", unique_zipcodes)

In [ ]:
population_df = pd.read_csv('population_data.csv')

##Unemployment Rate (Web Scraping)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table', id='comp')
    return pd.read_html(str(table))[0] if table else None

def get_all_page_links(soup):
    pagination_div = soup.find('div', class_='pagination')
    links = pagination_div.find_all('a') if pagination_div else []
    return ["https://zipatlas.com" + link['href'] for link in links]

base_url = "https://zipatlas.com/us/ca/zip-code-comparison/highest-unemployment-rate.htm"
all_data = pd.DataFrame()

response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

# Start by scraping the first page
df = get_data_from_html(response.content)
if df is not None:
    all_data = pd.concat([all_data, df], ignore_index=True)

# Get all page links from the first page's pagination
page_links = get_all_page_links(soup)

# Iterate through all page links and scrape table data
for page_link in page_links:
    response = requests.get(page_link)
    if response.status_code == 200:
        df = get_data_from_html(response.content)
        if df is not None:
            all_data = pd.concat([all_data, df], ignore_index=True)
            print(f"Scraped data from {page_link}")
        else:
            print(f"No table found on {page_link}")
    else:
        print(f"Failed to load {page_link}")

In [ ]:
all_data.to_csv('unemployment_rates.csv', index=False)


In [ ]:
unemployment_rates_df=pd.read_csv('/content/unemployment_rates.csv')

In [ ]:
fin_data_df = pd.merge(population_df, unemployment_rates_df, on='Zip Code', how='inner')

# Display the final merged dataframe
fin_data_df.head()

In [ ]:
fin_data_df.to_csv('fin_data.csv', index=False)
